In [ ]:
%pip install transformers[torch] huggingface_hub datasets evaluate

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 7.7 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: dill
    Found existing installation: dill 0.3.9
    Uninstalling dill-0.3.9:
      Successfully uninstalled dill-0.3.9

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Torchtext není k dispozici pro poslední verzi pytorch, budeme tedy využuívat něco jiného ...


In [1]:
from transformers import Trainer, BertForSequenceClassification
from datasets import load_from_disk

import torch
import base

In [2]:
base.reset_seed()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [4]:
train = load_from_disk('./data/sst2/train-logits')
eval = load_from_disk('./data/sst2/eval-logits')
test = load_from_disk('./data/sst2/test-logits')

In [5]:
base.reset_seed()

In [6]:
model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
#model = base.freeze_model(model)

In [8]:
training_args = base.get_training_args(output_dir="./results/bertTest", logging_dir="./logs/bertTest", batch_size=128, epochs=10)

In [9]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.618700,0.544986,0.724771,0.731224,0.723120,0.721807
2,0.440000,0.479837,0.778670,0.780493,0.777858,0.777910
3,0.356300,0.460689,0.793578,0.793519,0.793593,0.793539
4,0.313500,0.484178,0.790138,0.791111,0.789541,0.789673
5,0.284700,0.472208,0.798165,0.798138,0.798013,0.798059
6,0.267100,0.485169,0.800459,0.802381,0.799676,0.799800
7,0.253900,0.472437,0.809633,0.809585,0.809527,0.809552
8,0.243600,0.477627,0.807339,0.807281,0.807359,0.807303
9,0.237000,0.482812,0.806193,0.806134,0.806106,0.806119
10,0.231800,0.488821,0.805046,0.805066,0.804854,0.804922


TrainOutput(global_step=4740, training_loss=0.3246717428859276, metrics={'train_runtime': 124.192, 'train_samples_per_second': 4880.669, 'train_steps_per_second': 38.167, 'total_flos': 96261626803200.0, 'train_loss': 0.3246717428859276, 'epoch': 10.0})

In [11]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [12]:
trainer.evaluate(test)

{'eval_loss': 0.257160484790802,
 'eval_accuracy': 0.8999257609502599,
 'eval_precision': 0.8980010348279385,
 'eval_recall': 0.8998568689241466,
 'eval_f1': 0.8988170705611727,
 'eval_runtime': 3.9333,
 'eval_samples_per_second': 1712.281,
 'eval_steps_per_second': 13.475,
 'epoch': 10.0}

In [13]:
base.reset_seed()

In [14]:
student_model = BertForSequenceClassification.from_pretrained("google/bert_uncased_L-2_H-128_A-2", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
training_args = base.get_training_args(output_dir="./results/bertTestDistil", logging_dir="./logs/bertTestDistil", remove_unused_columns=False, batch_size=128, epochs=10, temp=5, lambda_param=.5)

In [16]:
trainer = base.ImageDistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval,
    compute_metrics=base.compute_metrics,
)

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.056200,2.293038,0.723624,0.727039,0.722373,0.721779
2,2.165300,1.866549,0.780963,0.782600,0.780195,0.780269
3,1.633300,1.758314,0.787844,0.787826,0.787667,0.787721
4,1.347800,1.775770,0.788991,0.790039,0.788372,0.788500
5,1.179200,1.709650,0.795872,0.795843,0.795719,0.795764
6,1.075200,1.675385,0.794725,0.795169,0.794298,0.794430
7,1.009500,1.678985,0.802752,0.803122,0.803065,0.802751
8,0.958700,1.675722,0.808486,0.808422,0.808485,0.808444
9,0.919100,1.690499,0.805046,0.804980,0.804980,0.804980
10,0.899500,1.694093,0.805046,0.805187,0.804770,0.804872


TrainOutput(global_step=4740, training_loss=1.424363244937945, metrics={'train_runtime': 128.91, 'train_samples_per_second': 4702.038, 'train_steps_per_second': 36.77, 'total_flos': 96261626803200.0, 'train_loss': 1.424363244937945, 'epoch': 10.0})

In [18]:
student_model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-1

In [19]:
trainer.evaluate(test)

{'eval_loss': 0.9574729800224304,
 'eval_accuracy': 0.9024498886414254,
 'eval_precision': 0.9006456209092124,
 'eval_recall': 0.9021193123630606,
 'eval_f1': 0.9013158004779819,
 'eval_runtime': 3.9885,
 'eval_samples_per_second': 1688.621,
 'eval_steps_per_second': 13.288,
 'epoch': 10.0}